In [1]:
import argparse

import numpy as np
import pandas as pd
import scipy.sparse as sp
from sklearn.model_selection import KFold

In [2]:
%load_ext autoreload
%autoreload 2

from load_data import load_data
from model import GModel
from myutils import roc_auc, translate_result
from optimizer import Optimizer
from sampler import BalancedSampler

In [3]:
class Args:
    def __init__(self):
        self.device = "cpu"  # cuda:number or cpu
        self.data = "ctrp"  # Dataset{gdsc or ccle}

args = Args()
device = args.device

In [6]:
res, drug_feature, exprs, mut, cna, null_mask = load_data(args)

response matrix (res) shape: (470, 807)
exprs shape: (1089, 19851)
mut shape: (1089, 1667)
cna shape: (1090, 23316)
807
exprs shape: (807, 19851)
mut shape: (807, 1667)
cna shape: (807, 23316)
drug_feature shape: (470, 920)
response matrix (res) shape: (807, 470)
null_mask shape: (807, 470)


In [7]:
res

,16-beta-bromoandrosterone,3-Cl-AHPC,968,A-804598,AA-COCF3,ABT-199,ABT-737,AC55649,AGK-2,AM-580,...,trifluoperazine,triptolide,tubastatin A,valdecoxib,vandetanib,veliparib,vincristine,vorapaxar,vorinostat,zebularine
22Rv1,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23132/87,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
253J,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
253J-BV,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
42-MG-BA,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
YH-13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
YKG1,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
ZR-75-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
ZR-75-30,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0.0


In [8]:
adj_coo = sp.coo_matrix(res)
pos_edges = np.vstack((adj_coo.row, adj_coo.col)).T

# ネガティブエッジ候補作成
adj_dense = adj_coo.toarray()

# ポジティブ（接続ありかつ null_mask==0）
pos_mask = np.logical_and(adj_dense == 1, null_mask == 0)
pos_row, pos_col = np.where(pos_mask)
pos_edges = np.vstack((pos_row, pos_col)).T
pos_labels = np.ones(len(pos_edges), dtype=int)

# ネガティブ（接続なしかつ null_mask==0）
neg_mask = np.logical_and(adj_dense == 0, null_mask == 0)
neg_row, neg_col = np.where(neg_mask)
neg_edges = np.vstack((neg_row, neg_col)).T
neg_labels = np.zeros(len(neg_edges), dtype=int)

# 統合
all_edges = np.vstack((pos_edges, neg_edges))
all_labels = np.concatenate((pos_labels, neg_labels))

In [9]:
exprs.shape

(807, 19851)

In [10]:
len(all_labels)

46313

In [12]:
k = 5
kf = KFold(n_splits=5, shuffle=True, random_state=42)

true_datas = pd.DataFrame()
predict_datas = pd.DataFrame()

for i, (train_idx, test_idx) in enumerate(kf.split(all_edges)):
    train_edges = all_edges[train_idx]
    train_labels = all_labels[train_idx]
    test_edges = all_edges[test_idx]
    test_labels = all_labels[test_idx]

    sampler = BalancedSampler(
        edge_train=train_edges,
        label_train=train_labels,
        edge_test=test_edges,
        label_test=test_labels,
        adj_shape=res.shape,
    )
    
    model = GModel(
        adj_mat=sampler.train_data,
        gene=exprs,
        cna=cna,
        mutation=mut,
        sigma=2,
        k=11,
        iterates=3,
        feature_drug=drug_feature,
        n_hid1=192,
        n_hid2=36,
        alpha=5.74,
        device=device,
    )
    opt = Optimizer(
        model,
        sampler.train_data,
        sampler.test_data,
        sampler.test_mask,
        sampler.train_mask,
        roc_auc,
        lr=5e-4,
        epochs=1000,
        device=device,
    ).to(device)
    epoch, true_data, predict_data = opt()
    true_datas = pd.concat([true_datas, translate_result(true_data)])
    predict_datas = pd.concat([predict_datas, translate_result(predict_data)])

epoch:   0 loss:0.808333 auc:0.4774
epoch:  20 loss:0.457782 auc:0.8866
epoch:  40 loss:0.383677 auc:0.9005
epoch:  60 loss:0.362651 auc:0.9056
epoch:  80 loss:0.348788 auc:0.9097
epoch: 100 loss:0.329075 auc:0.9153
epoch: 120 loss:0.305943 auc:0.9193
epoch: 140 loss:0.274550 auc:0.9226
epoch: 160 loss:0.237769 auc:0.9270
epoch: 180 loss:0.208660 auc:0.9333
epoch: 200 loss:0.190940 auc:0.9372
epoch: 220 loss:0.179287 auc:0.9399
epoch: 240 loss:0.170774 auc:0.9419
epoch: 260 loss:0.164325 auc:0.9435
epoch: 280 loss:0.159168 auc:0.9447
epoch: 300 loss:0.155114 auc:0.9459
epoch: 320 loss:0.151652 auc:0.9468
epoch: 340 loss:0.149146 auc:0.9474
epoch: 360 loss:0.147992 auc:0.9477
epoch: 380 loss:0.144579 auc:0.9487
epoch: 400 loss:0.142944 auc:0.9491
epoch: 420 loss:0.140966 auc:0.9496
epoch: 440 loss:0.140030 auc:0.9498
epoch: 460 loss:0.138457 auc:0.9504
epoch: 480 loss:0.137304 auc:0.9506
epoch: 500 loss:0.136350 auc:0.9508
epoch: 520 loss:0.135880 auc:0.9509
epoch: 540 loss:0.134743 auc

In [17]:
true_datas.to_csv(f"true_{args.data}.csv")
predict_datas.to_csv(f"pred_{args.data}.csv")